In [1]:
import pandas as pd
import numpy as np

In [2]:
songs_df = pd.read_csv('Data/songs_w_genres_df.csv', index_col='id')
#songs_df.head()

In [3]:
songs_df.genres = songs_df.genres.str[1:-1]
#songs_df.head()

In [4]:
songs_df = songs_df.astype({'acousticness': 'float32', 'danceability': 'float32', 'energy': 'float32', 
                            'instrumentalness': 'float32', 'liveness': 'float32', 'loudness': 'float32', 
                            'speechiness': 'float32', 'tempo': 'float32', 'valence': 'float32',})

In [5]:
#songs_df.info()

In [6]:
top_5_df = pd.read_csv('Data/top_5_df.csv')
top_5_df.rename(columns={'Unnamed: 0':'id'}, inplace=True)
top_5_df.reset_index(inplace=True)
top_5_df.drop('index', axis=1, inplace=True)
top_5_df.set_index('id', inplace=True)
#top_5_df

In [7]:
songs_1980s_up = songs_df[songs_df['year'] > 1980]
#songs_1980s_up.year.unique()

In [8]:
songs_1980s_up.reset_index(inplace=True)
songs_1980s_up.set_index(['id','name'], inplace=True)

In [9]:
numerical_songs = songs_1980s_up.select_dtypes(include=['float32'])
#numerical_songs.info()

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(numerical_songs), columns=numerical_songs.columns, index=numerical_songs.index)
#df_scaled.head()

In [12]:
indices = pd.Series(df_scaled.index.get_level_values(1))
#indices

In [13]:
# Cosine Similarity

In [14]:
songs_1980s_up_scaled = df_scaled[['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 
                                        'loudness', 'speechiness', 'tempo', 'valence']]
cosine_sim = cosine_similarity(songs_1980s_up_scaled, songs_1980s_up_scaled)
#cosine_sim

In [15]:
#TRY TO GET DROP DOWN LIST WITH ARTIST NAME FOR SONGS WITH SAME NAME
def num_of_artists(name):
    artists = []
    name = name.lower()
    for i in range(len(indices[indices==name])):
        idx = indices[indices == name].index[i]
        artists.append(songs_1980s_up.iloc[idx][0])
    return artists

In [16]:
def recommendations(name, artists, cosine_sim=cosine_sim):
    #INPUT A SONG RETURN
    name = name.lower()
    ind = 0
    for i in range(len(indices[indices == name])):
        if(songs_1980s_up.iloc[indices[indices == name].index[i]][0] == artists):
            ind = i
    idx = indices[indices == name].index[ind]
    print(songs_1980s_up.iloc[idx][0])
    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:6]
    
    songs_index = [i[0] for i in similarity_scores]
    
    return songs_1980s_up.iloc[songs_index, [0]]

In [17]:
#recommendations('snowman', 'Sia', cosine_sim)

In [18]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets

from IPython.display import display, clear_output

In [1]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable --sys-prefix voila

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling: voila
- Writing config: C:\Users\vtkurt13\anaconda3\etc\jupyter
    - Validating...
      voila 0.2.7 ok


In [20]:
song_name = widgets.Text(placeholder='Please enter a song')

In [21]:
artists = num_of_artists(song_name.value)

In [22]:
artist = widgets.Dropdown(
            options=artists,
            description='Artists Name',
            disbaled=False)

In [23]:
def update_song(change):
    song_name.value = change['new']
    artists = num_of_artists(song_name.value)
    artist.options = artists
song_name.observe(update_song, 'value')


In [24]:
get_top_5 = widgets.Button(
                description='Find recommended songs',
                tooltip='Find Song',
                layout=widgets.Layout(width='30%'),
                style={'description_width': 'intitial'}
            )
output = widgets.Output(layout=widgets.Layout(width='100%'))

def on_button_clicked(event):
    with output:
        clear_output()
        top_5 = recommendations(str(song_name.value), str(artist.value), cosine_sim)
        top_5.reset_index(inplace=True)
        top_5['url'] = ''
        for i in range(len(top_5)):
            url = f'https://open.spotify.com/track/{top_5.iloc[i][0]}'
            #print(url)
            top_5.url[i] = url
        print(f'Here are the top 5 recommended songs for {song_name.value} by {artist.value}')
        print(top_5)
        
get_top_5.on_click(on_button_clicked)

vbox_result = widgets.VBox([get_top_5, output])

In [25]:
hbox_result = widgets.HBox([song_name, artist])

In [26]:
text_0 = widgets.HTML(value='<h1>Jacob\'s Jukebox<h1>', layout=widgets.Layout(align_content='center'))
text_1 = widgets.HTML(value='<h2>Enter in a song to find the top 5 recommended songs!')

vbox_text = widgets.VBox([text_0, text_1, hbox_result, vbox_result], layout=widgets.Layout(align_content='center'))

In [27]:
page = widgets.HBox([vbox_text], layout=widgets.Layout(align_content= 'center'))
display(page)